In [1]:
using OceananigansLagrangianFilter
using OceananigansLagrangianFilter: jld2_to_netcdf
using CairoMakie
using CUDA
using Oceananigans.TimeSteppers: reset!
using Printf
using OceananigansLagrangianFilter: LagrangianFilter, create_input_data_on_disk, load_data, create_original_vars, create_filtered_vars, initialise_filtered_vars, create_forcing, create_output_fields, update_input_data!

In [2]:
config = OfflineFilterConfig(original_data_filename="SW_vort_with_tracer.jld2", 
                                    var_names_to_filter = ("ω", "T"), 
                                    velocity_names = ("u","v"),
                                    architecture = GPU(),
                                    Δt = 1e-3,
                                    T_out=0.1,
                                    T = 5,
                                    N=1,
                                    freq_c = 1,
                                    output_netcdf = true,
                                    delete_intermediate_files = false)


Setting filter parameters to use Butterworth squared, order 2, cutoff frequency 1


OfflineFilterConfig("SW_vort_with_tracer.jld2", ("ω", "T"), ("u", "v"), 0.0, 5.0, 5, GPU{CUDABackend}(CUDABackend(false, true)), 0.1, (a1 = 0.35355339059327373, b1 = 0.3535533905932738, c1 = 0.7071067811865475, d1 = 0.7071067811865476, N_coeffs = 1), 0.001, InMemory{Int64}(1, 4), true, "forward_output.jld2", "backward_output.jld2", "combined_output.jld2", false, 5, false, true, WENO{3, Float64, Float32}(order=5)
├── buffer_scheme: WENO{2, Float64, Float32}(order=3)
└── advection_velocity_scheme: Centered(order=4), 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo
├── Periodic x ∈ [-1.91418e-18, 6.28319) regularly spaced with Δx=0.0245437
├── Periodic y ∈ [-1.91418e-18, 6.28319) regularly spaced with Δy=0.0245437
└── Flat z                               )

In [ ]:
run_offline_Lagrangian_filter(config)

In [ ]:
# Copy and manipulate data on disk to have correct order and time shift
create_input_data_on_disk(config; direction = "forward") 

# Load in saved data from simulation
# this is a silly way to do it but it works for now
saved_velocities, saved_original_vars = load_data(config)
println("Loaded data from $(config.original_data_filename)")

# Create the original variables - these will be auxiliary fields in the model
original_vars = create_original_vars(config)
println("Created original variables: ", original_vars)

# Create the filtered variables - these will be tracers in the model
filtered_vars = create_filtered_vars(config)
println("Created filtered variables: ", filtered_vars)

# Create forcing for these filtered variables
forcing = create_forcing(filtered_vars, config)
println("Created forcing for filtered variables ")

# Define model 
model = LagrangianFilter(;config.grid, tracers = filtered_vars, auxiliary_fields = original_vars, forcing = forcing, advection=config.advection)

println("Created model")

# We can set initial values to improve the spinup, use the limit freq_c -> \infty
#The map variables get automatically initialised to zero
initialise_filtered_vars(model, saved_original_vars, config)    
println("Initialised filtered variables")

# Define our outputs # 
filtered_outputs = create_output_fields(model, config)
println("Defined outputs")

# Define the filtering simulation 
simulation = Simulation(model, Δt = config.Δt, stop_time = config.T) 
println("Defined simulation")

# Tell the simulation to use the saved data
simulation.callbacks[:update_input_data] = Callback(update_input_data!, parameters = (velocities = saved_velocities, original_vars = saved_original_vars))

# Add a progress monitor
function progress(sim)
    @info @sprintf("Simulation time: %s, max ω_C1: %.2e, max ω: %.2e\n ", 
                prettytime(sim.model.clock.time),minimum(model.tracers.ω_C1), maximum(model.auxiliary_fields.ω))             
    return nothing
end

simulation.callbacks[:progress] = Callback(progress,TimeInterval(config.T_out))

# filtered_outputs["u"] = model.velocities.u
# filtered_outputs["v"] = model.velocities.v
# filtered_outputs["ω_C1"] = model.tracers.ω_C1
# filtered_outputs["ω_S1"] = model.tracers.ω_S1
# filtered_outputs["T_C1"] = model.tracers.T_C1
# filtered_outputs["T_S1"] = model.tracers.T_S1
# filtered_outputs["ω"] = model.auxiliary_fields.ω
# filtered_outputs["T"] = model.auxiliary_fields.T
# filtered_outputs["xi_u_C1"] = model.tracers.xi_u_C1
# filtered_outputs["xi_v_C1"] = model.tracers.xi_v_C1
# filtered_outputs["xi_u_S1"] = model.tracers.xi_u_S1
# filtered_outputs["xi_v_S1"] = model.tracers.xi_v_S1
# filtered_outputs["T"] = model.auxiliary_fields.T

#Write outputs
simulation.output_writers[:vars] = JLD2Writer(model, filtered_outputs,
                                                        filename = config.forward_output_filename,
                                                        schedule = TimeInterval(config.T_out),
                                                        overwrite_existing = true)

# Run forward simulation                                                        
run!(simulation)

Loaded data from SW_vort_with_tracer.jld2
Created original variables: NamedTuple with 2 Fields on 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo:
├── T: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
└── ω: 256×256×1 Field{Center, Center, Center} on RectilinearGrid on CUDAGPU
Created filtered variables: (:ω_C1, :T_C1, :xi_u_C1, :xi_v_C1, :ω_S1, :T_S1, :xi_u_S1, :xi_v_S1)
Created model
Initialised filtered variables
Defined outputs
Defined simulation


[ Info: Initializing simulation...
┌ Info: Simulation time: 0 seconds, max ω_C1: -9.13e-01, max ω: 1.19e+00
└  
[ Info:     ... simulation initialization complete (37.236 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (9.305 seconds).
┌ Info: Simulation time: 100 ms, max ω_C1: -9.10e-01, max ω: 1.47e+00
└  
┌ Info: Simulation time: 200 ms, max ω_C1: -9.08e-01, max ω: 1.69e+00
└  
┌ Info: Simulation time: 300.000 ms, max ω_C1: -9.06e-01, max ω: 1.84e+00
└  
┌ Info: Simulation time: 400 ms, max ω_C1: -9.01e-01, max ω: 1.96e+00
└  
┌ Info: Simulation time: 500 ms, max ω_C1: -9.02e-01, max ω: 1.98e+00
└  
┌ Info: Simulation time: 600.000 ms, max ω_C1: -8.99e-01, max ω: 1.93e+00
└  
┌ Info: Simulation time: 700.000 ms, max ω_C1: -9.00e-01, max ω: 1.74e+00
└  
┌ Info: Simulation time: 800 ms, max ω_C1: -8.99e-01, max ω: 1.45e+00
└  
┌ Info: Simulation time: 900 ms, max ω_C1: -8.98e-01, max ω: 1.28e+00
└  
┌ Info: Simulation time: 1 second, max ω_C1

In [11]:
config.advection

WENO{3, Float64, Float32}(order=5)
├── buffer_scheme: WENO{2, Float64, Float32}(order=3)
└── advection_velocity_scheme: Centered(order=4)

In [4]:
jld2_to_netcdf(config.forward_output_filename, joinpath(@__DIR__, "forward_output_debug.nc"))


Wrote NetCDF file to /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/turbulent_SW/forward_output_debug.nc


In [2]:
jld2_to_netcdf("SW_vort_with_tracer_filter_input.jld2", joinpath(@__DIR__, "SW_vort_with_tracer_filter_input.nc"))

Wrote NetCDF file to /home/lbaker/Documents/Projects/OceananigansLagrangianFilter/testing/turbulent_SW/SW_vort_with_tracer_filter_input.nc


In [8]:
model.forcing.ω_S1

zeroforcing (generic function with 1 method)

In [47]:
model.forcing

(u = Oceananigans.Forcings.zeroforcing, v = Oceananigans.Forcings.zeroforcing, w = Oceananigans.Forcings.zeroforcing, ω_C1 = Oceananigans.Forcings.zeroforcing, T_C1 = Oceananigans.Forcings.zeroforcing, xi_u_C1 = Oceananigans.Forcings.zeroforcing, xi_v_C1 = Oceananigans.Forcings.zeroforcing, ω_S1 = Oceananigans.Forcings.zeroforcing, T_S1 = Oceananigans.Forcings.zeroforcing, xi_u_S1 = Oceananigans.Forcings.zeroforcing, xi_v_S1 = Oceananigans.Forcings.zeroforcing, T = Oceananigans.Forcings.zeroforcing, ω = Oceananigans.Forcings.zeroforcing)

In [31]:
# Animate
timeseries1 = FieldTimeSeries(config.forward_output_filename, "ω")
timeseries2 = FieldTimeSeries(config.forward_output_filename, "ω_filtered")
timeseries3 = FieldTimeSeries(config.forward_output_filename, "ω_filtered")
times = timeseries1.times

set_theme!(Theme(fontsize = 20))
fig = Figure(size = (1000, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "y",
               limits = ((0, 2π), (0, 2π)),
               aspect = AxisAspect(1))

ax1 = Axis(fig[2, 1]; title = "Raw vorticity", axis_kwargs...)
ax2 = Axis(fig[2, 2]; title = "Filtered vorticity", axis_kwargs...)
ax3 = Axis(fig[2, 3]; title = "Filtered regridded vorticity", axis_kwargs...)


n = Observable(1)
Observable(1)

var1 = @lift timeseries1[$n]
var2 = @lift timeseries2[$n]
var3 = @lift timeseries3[$n]

heatmap!(ax1, var1; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax2, var2; colormap = :balance, colorrange = (-1, 1))
heatmap!(ax3, var3; colormap = :balance, colorrange = (-1, 1))


title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:3], title, fontsize=24, tellwidth=false)

fig

frames = 1:length(times)

@info "Making an animation"

CairoMakie.record(fig, "LF_movie_DEBUG.mp4", frames, framerate=24) do i
    n[] = i
end

[ Info: Making an animation


"LF_movie_DEBUG.mp4"

In [46]:
filtered_vars

(:ω_C1, :T_C1, :xi_u_C1, :xi_v_C1, :ω_S1, :T_S1, :xi_u_S1, :xi_v_S1)

In [48]:
config

OfflineFilterConfig("SW_vort_with_tracer.jld2", ("ω", "T"), ("u", "v"), 0.0, 5.0, 5, GPU{CUDABackend}(CUDABackend(false, true)), 0.1, (a1 = 0.35355339059327373, b1 = 0.3535533905932738, c1 = 0.7071067811865475, d1 = 0.7071067811865476, N_coeffs = 1), 0.001, InMemory{Int64}(1, 4), true, "forward_output.jld2", "backward_output.jld2", "combined_output.jld2", false, 5, false, true, WENO{3, Float64, Float32}(order=5)
├── buffer_scheme: WENO{2, Float64, Float32}(order=3)
└── advection_velocity_scheme: Centered(order=4), 256×256×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on CUDAGPU with 3×3×0 halo
├── Periodic x ∈ [-1.91418e-18, 6.28319) regularly spaced with Δx=0.0245437
├── Periodic y ∈ [-1.91418e-18, 6.28319) regularly spaced with Δy=0.0245437
└── Flat z                               )